# E-commerce Brasileiro (Olist)

In [ ]:
import pandas as pd
import numpy as np

# 1. Carregar as tabelas principais
orders = pd.read_csv('olist_orders_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')

# 2. Unir as tabelas (Merge)
# Unindo Pedidos com Itens
df = pd.merge(orders, items, on='order_id', how='inner')

# Unindo com as Avaliações (Importante para NLP e Classificação)
df = pd.merge(df, reviews, on='order_id', how='inner')

# Unindo com Detalhes dos Produtos (Importante para Regressão/Categorias)
df = pd.merge(df, products, on='product_id', how='inner')

# 3. Criando variáveis úteis (Feature Engineering)
# Converter datas para o formato correto
date_cols = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col])

# Criar coluna de "Tempo de Entrega Real" (em dias)
df['delivery_time'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days

# Criar coluna de "Atraso" (Diferença entre estimada e real)
df['is_late'] = (df['order_delivered_customer_date'] > df['order_estimated_delivery_date']).astype(int)

# 4. Preparar Alvos (Targets) para o Trabalho

# Para Classificação: 1 se a nota for alta (4 ou 5), 0 se for baixa (1, 2 ou 3)
# O professor pediu para tratar desbalanceamento, e notas baixas são raras aqui!
df['target_class'] = df['review_score'].apply(lambda x: 1 if x >= 4 else 0)

# Para NLP: Filtrar apenas onde existe comentário de texto
df_nlp = df.dropna(subset=['review_comment_message']).copy()

# 5. Salvar o arquivo final para o grupo usar
df.to_csv('olist_processado.csv', index=False)
print("Arquivo 'olist_processado.csv' gerado com sucesso!")